In [1]:
# Import librairies
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql import functions as F

#os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages=org.apache.hadoop:hadoop-aws:2.7.3 pyspark-shell"

In [2]:
config = configparser.ConfigParser()
#config.read('dl.cfg')
config.read_file(open('dl.cfg'))

os.environ["AWS_ACCESS_KEY_ID"]= config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"]= config['AWS']['AWS_SECRET_ACCESS_KEY']

In [3]:
# Create spark session
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .config("spark.hadoop.fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .getOrCreate()
        
        
    spark.sparkContext._jsc.hadoopConfiguration().set("mapreduce.fileoutputcommitter.algorithm.version", "2")
    #spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.eu-west-3.amazonaws.com")
    return spark
spark = create_spark_session()

In [5]:
# 1- Get bodyStyle data
bodyStyle =spark.read.parquet("s3a://lakestorage/bodyStyle/")

In [10]:
bodyStyle.createOrReplaceTempView("bodyStyle_temp")
spark.sql("""
select count(*)
from bodyStyle_temp 
""").show()

+--------+
|count(1)|
+--------+
|  266440|
+--------+



In [12]:
# 2- Get car data
car =spark.read.parquet("s3a://lakestorage/car/")
car.createOrReplaceTempView("car_temp")
spark.sql("""
select count(*)
from car_temp 
""").show()

+--------+
|count(1)|
+--------+
|  266440|
+--------+



In [13]:
# 3- Get carWarpedBox data
carWarpedBox =spark.read.parquet("s3a://lakestorage/carWarpedBox/")
carWarpedBox.createOrReplaceTempView("carWarpedBox_temp")
spark.sql("""
select count(*)
from carWarpedBox_temp 
""").show()

+--------+
|count(1)|
+--------+
|  266440|
+--------+



In [14]:
# 4- Get color data
color =spark.read.parquet("s3a://lakestorage/color/")
color.createOrReplaceTempView("color_temp")
spark.sql("""
select count(*)
from color_temp 
""").show()

+--------+
|count(1)|
+--------+
|  266440|
+--------+



In [15]:
# 5- Get country data
country =spark.read.parquet("s3a://lakestorage/country/")
country.createOrReplaceTempView("country_temp")
spark.sql("""
select count(*)
from country_temp 
""").show()

+--------+
|count(1)|
+--------+
|  266440|
+--------+



In [16]:
# 6- Get makeModelYear data
makeModelYear =spark.read.parquet("s3a://lakestorage/makeModelYear/")
makeModelYear.createOrReplaceTempView("makeModelYear_temp")
spark.sql("""
select count(*)
from makeModelYear_temp 
""").show()

+--------+
|count(1)|
+--------+
|  266440|
+--------+



In [17]:
# 7- Get plate data
plate_ =spark.read.parquet("s3a://lakestorage/plate/")
plate_.createOrReplaceTempView("plate_temp")
spark.sql("""
select count(*)
from plate_temp 
""").show()

+--------+
|count(1)|
+--------+
|  245155|
+--------+



In [18]:
# 8- Get plateWarpedBox data
plateWarpedBox =spark.read.parquet("s3a://lakestorage/plateWarpedBox/")
plateWarpedBox.createOrReplaceTempView("plateWarpedBox_temp")
spark.sql("""
select count(*)
from plateWarpedBox_temp 
""").show()

+--------+
|count(1)|
+--------+
|  266440|
+--------+

